# Machine Learning Course Project

* Muhammet Emin Yüce
* Recep Oğuzhan Çetin

In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Data Read

In [2]:
#DO NOT change this
KERNEL_SIZE = 3
# change path to Dataset directory which contains images
def ImRead(PICTURE_NUM):
    path = "C:/Users/ycema/Desktop/Machine Learning Project/ProjectTestImages"

    # Path addition
    if(PICTURE_NUM < 10):
        source_picture_name = "/p00"+ str(PICTURE_NUM)+ ", a_source.png"
        target_name = "/p00"+ str(PICTURE_NUM)+ ", b_target.png"
        ground_truth_name = "/p00"+ str(PICTURE_NUM)+ ", c_groundtruth.png"
    else:
        source_picture_name = "/p0"+ str(PICTURE_NUM)+ ", a_source.png"
        target_name = "/p0"+ str(PICTURE_NUM)+ ", b_target.png"
        ground_truth_name = "/p0"+ str(PICTURE_NUM)+ ", c_groundtruth.png"

    # Image read for Source which is y_train and it's grayscale conversion which is x_train, 
    # Target image which is X_test, Groundtruth which is y_test
    src_img = cv2.imread(path + source_picture_name) #y_train
    gray_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2GRAY) # X_train
    target_img = cv2.imread(path + target_name,cv2.IMREAD_GRAYSCALE) #x_test
    groundtruth_img = cv2.imread(path + ground_truth_name)

    target_dim = target_img.shape
    return src_img, gray_img, target_img, groundtruth_img

# Feature Extraction

In [3]:
# Sift Feature Extraction
def siftFeatureExt(img):
    # This Function requires pip install opencv-contrib-python for 4.5.1 , 
    # in case you don't have it, run pip uninstall opencv then run pip install opencv-contrib-python
    
    # This code is an implementation of 
    # https://medium.com/machine-learning-world/feature-extraction-and-similar-image-search-with-opencv-for-newbies-3c59796bf774
    sift = cv2.xfeatures2d.SIFT_create(edgeThreshold = 10)
    # For Keypoints
    kp = sift.detect(gray_img,None)
    # For Descriptors
    kps, dsc = sift.compute(gray_img, kp)

    # Making it sklearn compatible
    dsc = dsc.flatten()
    needed_size = img.shape[0]* img.shape[1]
    if dsc.size < needed_size:
        dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
    else:
        return dsc[:needed_size]
    return dsc

In [4]:
from scipy.signal import convolve2d
# This function creates all the kernel then convolves the actual input with them
# Which Results in features for the Model 
def ExtractFeatures(img):
    kernel_dim = (KERNEL_SIZE,KERNEL_SIZE)
    
    kernels = []
    
#     Laplacian kernel
#     Laplacian_kernel = np.array([[0,1,0],
#                                   [1,-4,1],
#                                   [0,1,1]])
#     Sobeloperator_kernel = np.array([[-1,0,1],
#                                  [-2,0,2],
#                                  [-1,0,1]])
#     Sobeloperator_kernel2 = np.array([[1,2,1],
#                                  [0,0,0],
#                                  [-1,-2,-1]])
#     kernels.append(Laplacian_kernel)#[1,0,0]
                                    #[0,0,0]
                                    #[0,0,0]
#     kernels.append(Sobeloperator_kernel)
#     kernels.append(Sobeloperator_kernel2)

    # 8 Neighbor Filters
    for i in range(KERNEL_SIZE):
        for j in range(KERNEL_SIZE):
            kernel = np.zeros(kernel_dim)
            kernel[i,j] = 1
            kernels.append(kernel)
        
    img_ft = np.float32(img)
    # Corner Harris Filter
    corner_out = cv2.cornerHarris(img_ft,2,3,0.04).flatten()
    
    
    # Generating the features by 2d convolution
    features = []
    features.append(img.flatten())
    for i in kernels:
        features.append(np.asarray(convolve2d(img,i, "same")).flatten())
    features.append(corner_out)
    features.append(siftFeatureExt(img))
    
    # Transpose the features to fit the sklearn input format
    features = np.transpose(features)
    return features

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel



def fitModel(X):
    start = time.time()
    
    # 3 Models for r,g,b channels
#     reg_b = DecisionTreeRegressor()
#     reg_g = DecisionTreeRegressor()
#     reg_r = DecisionTreeRegressor()
    
    # These Methods below grants better results but they take mush more time 
    # for 30 images test case they are Commented out
    
#     reg_b = RandomForestRegressor(n_estimators = 3)
#     reg_g = RandomForestRegressor(n_estimators = 3)
#     reg_r = RandomForestRegressor(n_estimators = 3)
    
    # Pipeline method LinearRegression for getting most important features, selected 15 because its fast and efficient model
    # Decision tree for the pixel value regression
    reg_b = Pipeline([
      ('feature_selection', SelectFromModel(LinearRegression(),max_features = 30)),
      ('Regression', RandomForestRegressor(n_estimators = 30,n_jobs = -1, warm_start = True))
    ])
    reg_g = Pipeline([
      ('feature_selection', SelectFromModel(LinearRegression(),max_features = 30)),
      ('Regression', RandomForestRegressor(n_estimators = 30,n_jobs = -1, warm_start = True))
    ])
    reg_r = Pipeline([
      ('feature_selection', SelectFromModel(LinearRegression(),max_features = 30)),
      ('Regression', RandomForestRegressor(n_estimators = 30,n_jobs = -1, warm_start = True))
    ])

    reg_b.fit(X, b_y)
    reg_g.fit(X, g_y)
    reg_r.fit(X, r_y)

    
    end2 = time.time()
    print("Train time:{0:.2f} sec.".format(end2-start))
    return reg_b,reg_g,reg_r

In [6]:
def predictColorizedImage(X_test,reg_b,reg_g,reg_r):

    X_test = polynomial_features.transform(X_test)
    b_yhat = reg_b.predict(X_test).astype(np.uint8).reshape(target_dim[0],target_dim[1])
    g_yhat = reg_g.predict(X_test).astype(np.uint8).reshape(target_dim[0],target_dim[1])
    r_yhat = reg_r.predict(X_test).astype(np.uint8).reshape(target_dim[0],target_dim[1])
    #print(b_yhat.shape,g_yhat.shape,r_yhat.shape)
    # Merge 3 regressors predictions
    yhat = cv2.merge((b_yhat,g_yhat,r_yhat))
    #print(yhat.shape)
    yhat = yhat.astype(np.uint8)
    return yhat

In [7]:
# Main Code
from sklearn.metrics import mean_absolute_error
minImageIndex = 1
maxImageIndex = 50

f = open("ProjectTestImagesResults.txt", "w")

start = time.time()
total_mae = 0
for i in range(minImageIndex,maxImageIndex + 1):
    # Image reading
    src_img, gray_img, target_img, groundtruth_img = ImRead(i)
    
    #Split the y_train so we can make 3 regressors
    b_y, g_y, r_y = cv2.split(src_img)
    b_y = np.asarray(b_y).flatten()
    g_y = np.asarray(g_y).flatten()
    r_y = np.asarray(r_y).flatten()
    
    # Extract Features for model input
    X = ExtractFeatures(gray_img)
    
    # Polynomial Features helps to get better output but requires more time
    # Commented out for 30 image test case
    # If you want to test with polynomial Features 
    # uncomment this area with the part that is in functionpredictColorizedImage
    
    polynomial_features = PolynomialFeatures(degree = 2)
    X = polynomial_features.fit_transform(X)
    r_b, r_g , r_r = fitModel(X)

    # Extract Features for test data input
    target_dim = target_img.shape
    X_test = ExtractFeatures(target_img)
    yhat = predictColorizedImage(X_test,r_b,r_g,r_r)
    
    mae = mean_absolute_error(np.array(groundtruth_img).flatten().astype(int),np.array(yhat).flatten().astype(int))
    print('Mae for Image '+str(i)+' is {0:.2f}'.format(mae))
    print()
    total_mae += mae
    #Show the images
#     cv2.imshow("Predicted img:"+str(i) , yhat)
    cv2.imwrite(str(i)+".jpg", yhat) 
    f.write(str(mae))
    f.write("\n")

f.write("\n")
f.write(str(total_mae/maxImageIndex))
f.close()
end2 = time.time()
# Wait for this print line executes before opening any image window or kernel dies thanks to opencv
print("Total Train time:{0:.2f}".format(end2-start))
print('END')
print('Average MAE:',total_mae/maxImageIndex)
# cv2.waitKey(0)
  
#closing all open windows  
# cv2.destroyAllWindows()  

Train time:5.79 sec.
Mae for Image 1 is 18.03

Train time:12.00 sec.
Mae for Image 2 is 17.33

Train time:11.25 sec.
Mae for Image 3 is 15.42

Train time:7.03 sec.
Mae for Image 4 is 7.34

Train time:12.13 sec.
Mae for Image 5 is 9.05

Train time:13.67 sec.
Mae for Image 6 is 21.13

Train time:11.69 sec.
Mae for Image 7 is 16.67

Train time:6.39 sec.
Mae for Image 8 is 12.33

Train time:8.05 sec.
Mae for Image 9 is 16.36

Train time:6.77 sec.
Mae for Image 10 is 19.34

Train time:10.20 sec.
Mae for Image 11 is 11.81

Train time:7.14 sec.
Mae for Image 12 is 21.57

Train time:1.51 sec.
Mae for Image 13 is 33.41

Train time:24.95 sec.
Mae for Image 14 is 12.62

Train time:17.33 sec.
Mae for Image 15 is 27.60

Train time:1.86 sec.
Mae for Image 16 is 25.80

Train time:4.93 sec.
Mae for Image 17 is 15.43

Train time:17.36 sec.
Mae for Image 18 is 14.38

Train time:13.24 sec.
Mae for Image 19 is 16.56

Train time:4.72 sec.
Mae for Image 20 is 15.33

Train time:6.91 sec.
Mae for Image 21 is 